In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout, GRU
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np


In [ ]:
df = pd.read_csv('/content/df_full.csv')

df['LapTime'] = pd.to_timedelta(df['LapTime']).dt.total_seconds()

encoder = OneHotEncoder(sparse=False)
encoded_features = encoder.fit_transform(df[['Driver', 'FreshTyre']])

scaler = MinMaxScaler()
normalized_features = scaler.fit_transform(df[['DriverNumber', 'LapTime', 'Position']])

processed_features = np.hstack((normalized_features, encoded_features))

target = processed_features[:, 2]

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
time_steps = 10
samples = len(processed_features) // time_steps
processed_features_reshaped = processed_features[:samples * time_steps].reshape((samples, time_steps, -1))
target_reshaped = target[:samples * time_steps].reshape((samples, time_steps))

X_train, X_test, y_train, y_test = train_test_split(processed_features_reshaped, target_reshaped, test_size=0.2, random_state=42)




In [ ]:
model = Sequential()
model.add(SimpleRNN(50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=32)

In [ ]:
y_pred = model.predict(X_test)

y_pred_last = y_pred[:, -1]
y_test_last = y_test[:, -1]

mse = mean_squared_error(y_test_last, y_pred_last)
print("Mean Squared Error:", mse)

rmse = np.sqrt(mse)
print("Root Mean Squared Error:", rmse)

mae = mean_absolute_error(y_test_last, y_pred_last)
print("Mean Absolute Error:", mae)


130/130 [==============================] - 1s 3ms/step
Mean Squared Error: 4.006327848808438e-05
Root Mean Squared Error: 0.006329555947148614
Mean Absolute Error: 0.004477114570031544


In [ ]:
model2 = Sequential()
model2.add(GRU(500, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model2.add(Dropout(0.2))
model2.add(Dense(1))

model2.compile(optimizer='adam', loss='mean_squared_error')

model2.fit(X_train, y_train, epochs=10, batch_size=32)


Epoch 1/10
520/520 [==============================] - 17s 30ms/step - loss: 0.0063
Epoch 2/10
520/520 [==============================] - 16s 30ms/step - loss: 6.6019e-04
Epoch 3/10
520/520 [==============================] - 15s 29ms/step - loss: 4.7990e-04
Epoch 4/10
520/520 [==============================] - 15s 28ms/step - loss: 3.9683e-04
Epoch 5/10
520/520 [==============================] - 15s 29ms/step - loss: 3.4384e-04
Epoch 6/10
520/520 [==============================] - 15s 29ms/step - loss: 3.2220e-04
Epoch 7/10
520/520 [==============================] - 15s 29ms/step - loss: 2.8270e-04
Epoch 8/10
520/520 [==============================] - 16s 30ms/step - loss: 2.5638e-04
Epoch 9/10
520/520 [==============================] - 16s 31ms/step - loss: 2.2899e-04
Epoch 10/10
520/520 [==============================] - 16s 30ms/step - loss: 2.2032e-04


In [ ]:
y_pred = model2.predict(X_test)

y_pred_last = y_pred[:, -1]
y_test_last = y_test[:, -1]

mse = mean_squared_error(y_test_last, y_pred_last)
print("Mean Squared Error:", mse)

rmse = np.sqrt(mse)
print("Root Mean Squared Error:", rmse)

mae = mean_absolute_error(y_test_last, y_pred_last)
print("Mean Absolute Error:", mae)

130/130 [==============================] - 1s 7ms/step
Mean Squared Error: 3.240423655209215e-05
Root Mean Squared Error: 0.005692471919306423
Mean Absolute Error: 0.004333276637900551
